In [13]:
# Colab 目前使用 tensorflow 2.2 但也因此不能使用 keras 部分函式，故降版為 1.14.0
!pip install tensorflow==1.14.0
import tensorflow as tf
print(tf.__version__)

     |████████████████████████████████| 109.2MB 28kB/s 
     |████████████████████████████████| 491kB 76.9MB/s 
     |████████████████████████████████| 3.2MB 77.2MB/s 
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 1.12.2
    Uninstalling tensorboard-1.12.2:
      Successfully uninstalled tensorboard-1.12.2
  Found existing installation: tensorflow 1.12.0
    Uninstalling tensorflow-1.12.0:
      Successfully uninstalled tensorflow-1.12.0


2.2.0-rc2


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPooling2D, BatchNormalization
from keras import optimizers
ada = keras.optimizers.Adagrad
import matplotlib.pyplot as plt

In [0]:
batch_size = 64
num_classes = 10
epochs = 20

In [0]:
def build_model():
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
    model.add(Conv2D(filters=64, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Flatten())
    model.add(Dense(120, activation='relu'))
    model.add(Dense(84, activation = 'relu'))
    # 為輸入數據施加Dropout。Dropout將在訓練過程中每次更新參數時隨機斷開一定百分比（p）的輸入神經元連接，Dropout層用於防止過擬合。
    model.add(Dropout(rate=0.25))
    model.add(Dense(10, activation='softmax'))
    # For a multi-class classification problem, 而 optimizer 我使用 Adagrad
    model.compile(loss='categorical_crossentropy', 
                  optimizer=ada(lr=0.01), 
                  metrics=['accuracy'])
    return model

In [0]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
#show train img
print(x_train[0].shape) #32*32*3
plt.imshow(x_train[0])
#plt.savefig('train.png')
plt.clf()

In [0]:
#normalize img
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [0]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
#set model
model = build_model()
#model = load_model('lenet.h5')
model.summary()
# start training
# Train the model, iterating on the data in batches of 64 samples
history = model.fit(x_train[:40000], y_train[:40000],
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(x_train[40000:], y_train[40000:]),
                        shuffle=True)

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
predict = model.predict(x_test)
#model.save('lenet.h5')

In [0]:
print(predict.shape)
print(predict[0])

print(history.history.keys())
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [0]:
#plt.savefig('acc.png')
plt.clf()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
#plt.savefig('loss.png')